# Построение модели
### Выполнил: Земнухов Вадим. Группа: DS-10

### Содержание

[**Подготовка исходных данных для обучения и теста**](#id_1)

[**Построение архитектуры Encoder-Decoder**](#id_2)

[**Обучение модели**](#id_3)

[**Сохранение Encoder и Decoder моделей**](#id_4)

[**Проверка на единичных предсказаниях**](#id_5)

[**Предсказание для тестовой выборки**](#id_6)

[**Оценка качества модели и анализ ошибок**](#id_7)

[**Вывод**](#id_8)


<div id= 'id_1'/>

### Подготовка исходных данных для обучения и теста

In [3]:
import pandas as pd

In [16]:
df = pd.read_pickle('/kaggle/input/addresses/addresses_1.pkl')

In [3]:
df.head()

,address,target,Normalized
0,Улица Ремизова дом 1,улица ремизова дом 1,улица ремизова дом один
1,Улица Ремизова дом 2,улица ремизова дом 2,улица ремизова дом два
2,Улица Ремизова дом 3 корпус 1,улица ремизова дом 3 корпус 1,улица ремизова дом три корпус один
3,Улица Ремизова дом 3 корпус 2,улица ремизова дом 3 корпус 2,улица ремизова дом три корпус два
4,Улица Ремизова дом 4,улица ремизова дом 4,улица ремизова дом четыре


In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
df.shape

(102799, 3)

In [6]:
df['target'] = df['target'].apply(lambda x : 'START_ '+ x + ' _END')
df.head()

,address,target,Normalized
0,Улица Ремизова дом 1,START_ улица ремизова дом 1 _END,улица ремизова дом один
1,Улица Ремизова дом 2,START_ улица ремизова дом 2 _END,улица ремизова дом два
2,Улица Ремизова дом 3 корпус 1,START_ улица ремизова дом 3 корпус 1 _END,улица ремизова дом три корпус один
3,Улица Ремизова дом 3 корпус 2,START_ улица ремизова дом 3 корпус 2 _END,улица ремизова дом три корпус два
4,Улица Ремизова дом 4,START_ улица ремизова дом 4 _END,улица ремизова дом четыре


Соберем уникальные слова и числа в колонке, которой будет делаться предсказание и целевой.

In [7]:
all_words=set()
for norm in df['Normalized']:
    for word in norm.split():
        if word not in all_words:
            all_words.add(word)

all_targets=set()
for target in df['target']:
    for word in target.split():
        if word not in all_targets:
            all_targets.add(word)

In [8]:
print(len(all_words))

2580


In [9]:
print(len(all_targets))

3651


In [10]:
df['length_Normalized']=df['Normalized'].apply(lambda x:len(x.split(' ')))
df['length_target']=df['target'].apply(lambda x:len(x.split(' ')))

df.head()

,address,target,Normalized,length_Normalized,length_target
0,Улица Ремизова дом 1,START_ улица ремизова дом 1 _END,улица ремизова дом один,4,6
1,Улица Ремизова дом 2,START_ улица ремизова дом 2 _END,улица ремизова дом два,4,6
2,Улица Ремизова дом 3 корпус 1,START_ улица ремизова дом 3 корпус 1 _END,улица ремизова дом три корпус один,6,8
3,Улица Ремизова дом 3 корпус 2,START_ улица ремизова дом 3 корпус 2 _END,улица ремизова дом три корпус два,6,8
4,Улица Ремизова дом 4,START_ улица ремизова дом 4 _END,улица ремизова дом четыре,4,6


In [11]:
df['target'].iloc[0].split(' ')

['START_', 'улица', 'ремизова', 'дом', '1', '_END']

In [12]:
input_words = sorted(list(all_words))
target_words = sorted(list(all_targets))
num_encoder_tokens = len(all_words)
num_decoder_tokens = len(all_targets)
num_encoder_tokens, num_decoder_tokens

(2580, 3651)

In [13]:
num_decoder_tokens += 1 #padding

In [14]:
df

,address,target,Normalized,length_Normalized,length_target
0,Улица Ремизова дом 1,START_ улица ремизова дом 1 _END,улица ремизова дом один,4,6
1,Улица Ремизова дом 2,START_ улица ремизова дом 2 _END,улица ремизова дом два,4,6
2,Улица Ремизова дом 3 корпус 1,START_ улица ремизова дом 3 корпус 1 _END,улица ремизова дом три корпус один,6,8
3,Улица Ремизова дом 3 корпус 2,START_ улица ремизова дом 3 корпус 2 _END,улица ремизова дом три корпус два,6,8
4,Улица Ремизова дом 4,START_ улица ремизова дом 4 _END,улица ремизова дом четыре,4,6
...,...,...,...,...,...
103043,Проспект Мира дом 27 строение 9,START_ проспект мира дом 27 строение 9 _END,проспект мира дом двадцать семь строение девять,7,8
103044,Проспект Мира дом 27 строение 10,START_ проспект мира дом 27 строение 10 _END,проспект мира дом двадцать семь строение десять,7,8
103045,Проспект Мира дом 27 строение 11,START_ проспект мира дом 27 строение 11 _END,проспект мира дом двадцать семь строение одинн...,7,8
103046,Проспект Мира дом 28,START_ проспект мира дом 28 _END,проспект мира дом двадцать восемь,5,6


In [15]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [16]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

Перемешаем наш датасет.

In [17]:
df = shuffle(df, random_state = 42)
df.head(10)

,address,target,Normalized,length_Normalized,length_target
69774,Тверской бульвар дом 28,START_ тверской бульвар дом 28 _END,тверской бульвар дом двадцать восемь,5,6
15560,Шоссейная улица дом 2 Д строение 8,START_ шоссейная улица дом 2 д строение 8 _END,шоссейная улица дом два д строение восемь,7,9
90376,Проезд Нансена дом 5 строение 1,START_ проезд нансена дом 5 строение 1 _END,проезд нансена дом пять строение один,6,8
32282,Кленовый бульвар дом 5,START_ кленовый бульвар дом 5 _END,кленовый бульвар дом пять,4,6
36030,1-я Дубровская улица дом 15 строение 19,START_ 1-я дубровская улица дом 15 строение 19...,первая дубровская улица дом пятнадцать строени...,7,9
13879,Очаковское шоссе дом 3 строение 4,START_ очаковское шоссе дом 3 строение 4 _END,очаковское шоссе дом три строение четыре,6,8
11873,Проезд Стройкомбината дом 4 строение 3,START_ проезд стройкомбината дом 4 строение 3 ...,проезд стройкомбината дом четыре строение три,6,8
32142,2-я Карачаровская улица дом 14 А строение 6,START_ 2-я карачаровская улица дом 14 а строен...,вторая карачаровская улица дом четырнадцать а ...,8,10
59218,Балаклавский проспект дом 2 корпус 6,START_ балаклавский проспект дом 2 корпус 6 _END,балаклавский проспект дом два корпус шесть,6,8
657,Промышленная улица дом 10 строение 2,START_ промышленная улица дом 10 строение 2 _END,промышленная улица дом десять строение два,6,8


Разделим выборку на тренировочную и тестовую.

In [18]:
X, y = df['Normalized'], df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)
X_train.shape, X_test.shape

((71959,), (30840,))

In [19]:
max_length_src=max(df['length_Normalized'])
max_length_tar=max(df['length_target'])

Напишем функцию для генерации батча.

In [20]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):

    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t>0:

                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

<div id= 'id_2'/>

### Построение архитектуры Encoder-Decoder

In [23]:
latent_dim=300

In [24]:
num_decoder_tokens+=1

In [25]:
num_encoder_tokens

2580

In [26]:
num_decoder_tokens

3653

In [27]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [28]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    774300      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    1095900     ['input_2[0][0]']                
                                                                                              

<div id= 'id_3'/>

### Обучение модели

In [22]:
import tensorflow

In [32]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [33]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50


/tmp/ipykernel_32/2108290721.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


562/562 [==============================] - 209s 359ms/step - loss: 3.1080 - val_loss: 2.3812
Epoch 2/50
562/562 [==============================] - 196s 349ms/step - loss: 2.0910 - val_loss: 1.8711
Epoch 3/50
562/562 [==============================] - 195s 348ms/step - loss: 1.6446 - val_loss: 1.4233
Epoch 4/50
562/562 [==============================] - 197s 351ms/step - loss: 1.2109 - val_loss: 1.0386
Epoch 5/50
562/562 [==============================] - 202s 359ms/step - loss: 0.8761 - val_loss: 0.7695
Epoch 6/50
562/562 [==============================] - 199s 354ms/step - loss: 0.6750 - val_loss: 0.6229
Epoch 7/50
562/562 [==============================] - 200s 355ms/step - loss: 0.5371 - val_loss: 0.5040
Epoch 8/50
562/562 [==============================] - 201s 358ms/step - loss: 0.4231 - val_loss: 0.3992
Epoch 9/50
562/562 [==============================] - 196s 349ms/step - loss: 0.3288 - val_loss: 0.3113
Epoch 10/50
562/562 [==============================] - 197s 351ms/step - lo

In [42]:
model.save_weights('/kaggle/working/Weights.h5')

In [43]:
model.save('/kaggle/working/model_ITN.h5')

In [122]:
import keras

In [54]:
model.summary()

Model: "model"

__________________________________________________________________________________________________

 Layer (type)                   Output Shape         Param #     Connected to                     


 input_1 (InputLayer)           [(None, None)]       0           []                               

                                                                                                  

 input_2 (InputLayer)           [(None, None)]       0           []                               

                                                                                                  

 embedding (Embedding)          (None, None, 300)    774300      ['input_1[0][0]']                

                                                                                                  

 embedding_1 (Embedding)        (None, None, 300)    1095900     ['input_2[0][0]']                

                                                                                   

<div id= 'id_4'/>

### Сохранение Encoder и Decoder моделей

In [34]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Напишем функцию для обратной нормализации с использованием получившихся Encoder и Decoder моделей.

In [27]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [26]:
decoder_model.save('/kaggle/working/decoder.h5')
encoder_model.save('/kaggle/working/encoder.h5')

<div id= 'id_5'/>

### Проверка на единичных предсказаниях

Попробуем сделать несколько единичных предсказаний.

In [69]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)

In [70]:
k=-1

In [71]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 27ms/step

1/1 [==============================] - 0s 25ms/step

1/1 [==============================] - 0s 26ms/step

1/1 [==============================] - 0s 26ms/step

1/1 [==============================] - 0s 27ms/step

1/1 [==============================] - 0s 29ms/step

1/1 [==============================] - 0s 27ms/step

1/1 [==============================] - 0s 34ms/step

Нормализованный адрес: неманский проезд дом семь корпус один

Адрес:  неманский проезд дом 7 корпус 1 

Предсказанный адрес:  неманский проезд дом 7 корпус 1 


In [111]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)

In [112]:
k=-1

In [113]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 30ms/step

1/1 [==============================] - 0s 34ms/step

1/1 [==============================] - 0s 29ms/step

1/1 [==============================] - 0s 29ms/step

1/1 [==============================] - 0s 28ms/step

1/1 [==============================] - 0s 29ms/step

1/1 [==============================] - 0s 37ms/step

1/1 [==============================] - 0s 40ms/step

Нормализованный адрес: неманский проезд дом семь корпус один

Адрес:  неманский проезд дом 7 корпус 1 

Предсказанный адрес:  неманский проезд дом 7 корпус 1 


In [116]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 60ms/step

1/1 [==============================] - 0s 61ms/step

1/1 [==============================] - 0s 94ms/step

1/1 [==============================] - 0s 74ms/step

1/1 [==============================] - 0s 122ms/step

1/1 [==============================] - 0s 105ms/step

1/1 [==============================] - 0s 80ms/step

1/1 [==============================] - 0s 46ms/step

1/1 [==============================] - 0s 49ms/step

Нормализованный адрес: вторая улица машиностроения дом три строение три

Адрес:  2-я улица машиностроения дом 3 строение 3 

Предсказанный адрес:  2-я улица машиностроения дом 3 строение 3 


In [117]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 54ms/step

1/1 [==============================] - 0s 42ms/step

1/1 [==============================] - 0s 38ms/step

1/1 [==============================] - 0s 40ms/step

1/1 [==============================] - 0s 49ms/step

1/1 [==============================] - 0s 72ms/step

1/1 [==============================] - 0s 40ms/step

1/1 [==============================] - 0s 43ms/step

Нормализованный адрес: столешников переулок дом девять строение три

Адрес:  столешников переулок дом 9 строение 3 

Предсказанный адрес:  столешников переулок дом 9 строение 3 


In [118]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 69ms/step

1/1 [==============================] - 0s 76ms/step

1/1 [==============================] - 0s 107ms/step

1/1 [==============================] - 0s 120ms/step

1/1 [==============================] - 0s 108ms/step

1/1 [==============================] - 0s 77ms/step

1/1 [==============================] - 0s 87ms/step

1/1 [==============================] - 0s 251ms/step

Нормализованный адрес: пос малино первомайская улица дом двадцать пять

Адрес:  пос малино первомайская улица дом 25 

Предсказанный адрес:  пос малино первомайская улица дом 25 


In [119]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 92ms/step

1/1 [==============================] - 0s 43ms/step

1/1 [==============================] - 0s 51ms/step

1/1 [==============================] - 0s 43ms/step

1/1 [==============================] - 0s 38ms/step

1/1 [==============================] - 0s 39ms/step

1/1 [==============================] - 0s 89ms/step

1/1 [==============================] - 0s 24ms/step

1/1 [==============================] - 0s 28ms/step

Нормализованный адрес: зеленоградская улица дом дванадцать а строение восемь

Адрес:  зеленоградская улица дом 12 а строение 8 

Предсказанный адрес:  зеленоградская улица дом 12 а строение 8 


In [122]:
decoded_sentence[:-4].strip()

'зеленоградская улица дом 12 а строение 8'

Проверим на единичных предсказаниях, как работает наша модель после её загрузки из файлов. Загрузим encoder и decoder.

In [21]:
import keras

decoder_model = keras.models.load_model('/kaggle/input/models-itn/decoder.h5')
encoder_model = keras.models.load_model('/kaggle/input/models-itn/encoder.h5')

In [126]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 27ms/step
Нормализованный адрес: голубинская улица дом двадцать четыре корпус два
Адрес:  голубинская улица дом 24 корпус 2 
Предсказанный адрес:  голубинская улица дом 24 корпус 2 


In [27]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 32ms/step
Нормализованный адрес: бибиревская улица дом два строение четыре
Адрес:  бибиревская улица дом 2 строение 4 
Предсказанный адрес:  бибиревская улица дом 2 строение 4 


In [28]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 30ms/step
Нормализованный адрес: улица винокурова дом пять дробь шесть корпус один
Адрес:  улица винокурова дом 5 / 6 корпус 1 
Предсказанный адрес:  улица винокурова дом 5 / 6 корпус 1 


In [29]:
k+=1

(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Нормализованный адрес:', X_test[k:k+1].values[0])
print('Адрес:', y_test[k:k+1].values[0][6:-4])
print('Предсказанный адрес:', decoded_sentence[:-4])

1/1 [==============================] - 0s 33ms/step
Нормализованный адрес: малая семёновская улица дом девять строение десять
Адрес:  малая семёновская улица дом 9 строение 10 
Предсказанный адрес:  малая семёновская улица дом 9 строение 10 


Единичные предсказания после загрузки моделей из файлов тоже выполняются корректно. Проверим качество модели на всем датасете.

<div id= 'id_6'/>

### Предсказание для тестовой выборки

In [123]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)

predict = np.array([])
target = np.array([])
k=-1

for i in range(len(y_test)):
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    predict = np.append(predict, decoded_sentence[:-4].strip())
    k+=1
    target = np.append (target, y_test[k:k+1].values[0][6:-4].strip())

1/1 [==============================] - 0s 35ms/step


In [124]:
result = pd.DataFrame({'prediction' : predict, 'target' : target})
result.head(10)

,prediction,target
0,енисейская улица дом 7 строение 14,енисейская улица дом 7 строение 14
1,4-й красносельский переулок дом 5 строение 3,4-й красносельский переулок дом 5 строение 3
2,вагоноремонтная улица дом 25 б,вагоноремонтная улица дом 25 б
3,5-я улица ямского поля дом 27 строение 2,5-я улица ямского поля дом 27 строение 2
4,улица исаковского дом 8 корпус 1 строение 3,улица исаковского дом 8 корпус 1 строение 3
5,производственная улица дом 6 строение 48,производственная улица дом 6 строение 48
6,ленская улица дом 24,ленская улица дом 24
7,5-й донской проезд дом 21 корпус 15 строение 3,5-й донской проезд дом 21 корпус 15 строение 3
8,5-й монетчиковский переулок дом 8 / 10,5-й монетчиковский переулок дом 8 / 10
9,улица трофимова дом 2 / 1,улица трофимова дом 2 / 1


In [126]:
result.shape

(30840, 2)

In [127]:
y_test.shape

(30840,)

In [128]:
result.to_pickle('/kaggle/working/result.pkl')

In [5]:
result = pd.read_pickle('/kaggle/input/final-result/result.pkl')
result.drop_duplicates(inplace = True)
result.shape

(30840, 2)

<div id= 'id_7'/>

### Оценка качества модели и анализ ошибок

Проверять качество будем по метрике accuracy. Проверим какая доля ответов модели полностью совпадает с предсказанием, а также посмотрим, в каких случаях модель ошибается.

In [12]:
accuracy = result[result['prediction']==result['target']].shape[0]/result.shape[0]
print(f'Accuracy = {accuracy}')

Accuracy = 0.9736705577172503


In [61]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

mistakes = result[result['prediction']!=result['target']]

In [69]:
norm_list = np.array([])
for i in range(mistakes.shape[0]):
    norm_list = np.append(norm_list, df[df['target'] == mistakes['target'].iloc[i]]['Normalized'])

In [65]:
mistakes.shape

(812, 2)

In [70]:
norm_list.shape

(812,)

In [72]:
mistakes['Normalized'] = norm_list

/tmp/ipykernel_32/3325143667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mistakes['Normalized'] = norm_list


In [73]:
mistakes

,prediction,target,Normalized
7,15-й микрорайон корпус 32,15-й микрорайон корпус 1532,один пятый микрорайон корпус тысяч пятьсот тридцать два
73,большая косинская улица дом 16 корпус 2 строен,большая косинская улица дом 16 корпус 2 строение 2,большая косинская улица дом шестнадцать корпус два строение два
88,малая калужская улица дом 15 строение 19,малая калужская улица дом 15 строение 11,малая калужская улица дом пятнадцать строение одиннадцать
20,1-й переулок измайловского зверинца дом 10 стро,1-й переулок измайловского зверинца дом 10 строение 11,первый переулок измайловского зверинца дом десять строение одиннадцать
23,большой трёхсвятительский переулок дом 2 / 1 стро,большой трёхсвятительский переулок дом 2 / 1 строение 6,большой трёхсвятительский переулок дом два дробь один строение шесть
51,набережная новикова-прибоя дом 10 корпус 2 стро,набережная новикова-прибоя дом 10 корпус 2 строение 2,набережная новикова прибоя дом десять корпус два строение два
68,западная улица дом 4 строение 91,западная улица дом 4 строение 119,западная улица дом четыре строение сто девятнадцать
77,нижний сусальный тупик дом 3 строение 1,нижний таганский тупик дом 3 строение 1,нижний таганский тупик дом три строение один
90,улица мичуринский проспект олимпийская деревня,улица мичуринский проспект олимпийская деревня дом 23 корпус 2,улица мичуринский проспект олимпийская деревня дом двадцать три корпус два
106,постолстопальцево центральная улица дом 21 б стро,постолстопальцево центральная улица дом 21 б строение 14,постолстопальцево центральная улица дом двадцать один б строение четырнадцать


<div id= 'id_8'/>

### Вывод:

1) Получили хорошее качество модели со значением метрики accuracy = 0.9736705577172503

2) Количество ошибок довольно небольшое, при этом наиболее часто они встречаются в адресах, содержащих большое количество слов. Для решения этой проблемы необходимо увеличить количество данных, на которых обучается модель, сделав при этом акцент на более длинные адреса.

3) Также можно заметить, что в некоторых случаях модель неверно осуществила обратную нормализацию самих чисел. В основном это числа, которые редко встречаются в самих адресах, такие как 1000 или 800 и т.д. Числа довольно высокого значения редко применяются в обозначении номеров домов, строений или корпусов. Для решения этой проблемы также необходимо иметь больше данных для обучения.

4) Даже в случаях, где модель ошиблась можно заметить, что определенные патерны всё же были "переведены" верно, например, порядковые числительные (1-й, 2-й и т.п), номера домов типа "40 / 1".

5) Можно заметить, что в наиболее редких случаях причиной ошибки стали не очень корректные данные для обучения, однако такие случаи единичны. Например слово "двасти" вместо "двести", или "дом четвертого мая" (получено после нормализации значения "4-5"), или "строение n", или "дом тысяч двести" вместо "тысяча двести". Такие значения встречались очень редко в наших исходных данных, поэтому корректировка исходного датасета и переобучение модели не даст значительного прироста качества.

6) Учитывая довольно высокую разносортность исходных данных, а также высокое значение доли правильных ответов, можно сделать вывод, что получившаяся модель пригодна для дальнейшего практического применения и может быть улучшена, за счет дообучения на новых исходных данных.